In [25]:
import os
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

In [26]:
df = pd.read_csv("subset_50_recipes.csv")

In [27]:
df.head

<bound method NDFrame.head of     Unnamed: 0                                              Title  \
0         7354                           Apple-Frangipane Galette   
1         1049                   Eggplant Omelet (Tortang Talong)   
2         6989                                    Blue Cheese Dip   
3          243                                             Hawaij   
4        12825           Spiced Fritters with Maple-Vanilla Syrup   
5         8749               Honey-Mustard Chicken-Sausage Kebabs   
6         7921               Maple Cake with Maple Syrup Frosting   
7        12259                              Turkey Cream Puff Pie   
8         9420  Eel With Olives, Chiles, and Capers (Anguilla ...   
9        10331                                     Pork Cassoulet   
10        3376          Thai Coconut, Broccoli and Coriander Soup   
11        5416                                Garlicky Mayonnaise   
12        7207                 Turkey Sweet Potato Shepherd's Pie   
13  

In [39]:
def describe_dish(row):
    title = row.get("Title", "Unknown Title")
    ingredients = row.get("Cleaned_Ingredients", "")
    instructions = row.get("Instruction", "")

    if len(instructions) > 100:
        instructions_summary = instructions[:100] + "..."
    else:
        instructions_summary = instructions

    description = (
        f"{title} is a delicious dish made with key ingredients such as {ingredients}. "
        f"In brief, the preparation involves: {instructions_summary}. "
        f"Give it a try if you're looking for a satisfying, flavorful meal!"
    )
    
    return description

In [40]:
df["description"] = df.apply(describe_dish, axis=1)

In [41]:
df.to_csv("recipes_with_descriptions.csv", index=False)